To make sure I understood this correctly, I have to make sure everything works empirically. This tool a while with several AIs... Anyway, bottom line is that things seem to work. And for a certain $\gamma=\frac{N}{M}$ we shuold expect $s_{max}=1+
\sqrt{\gamma}$

In [86]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
N = 256
gamma_values = [128, 256]
num_Ts = 200000 

def generate_complex_matrix(N, M):
    """Generate a normalized complex Gaussian random matrix."""
    sigma = np.sqrt(2) / 2
    real_part = np.random.normal(0, sigma, size=(N, M))
    imag_part = np.random.normal(0, sigma, size=(N, M))
    A = (real_part + 1j * imag_part) / np.sqrt(M)  
    return A

def marchenko_pastur_svd_pdf(x, gamma):
    x_plus = (1 + np.sqrt(gamma)) ** 2
    x_minus = (1 - np.sqrt(gamma)) ** 2
    density = np.zeros_like(x)
    valid = (x ** 2 >= x_minus) & (x ** 2 <= x_plus)
    epsilon = 1e-10  # Small value to avoid division by zero
    density[valid] = (1 / (gamma*np.pi * np.maximum(x[valid], epsilon))) * np.sqrt((x_plus - x[valid] ** 2) * (x[valid] ** 2 - x_minus))
    
    if gamma > 1:
        density *= gamma
        
        # Add point mass at zero
        if x[0] == 0:
            density[0] = (gamma - 1) / gamma

    
    return density

fig, axs = plt.subplots(1, len(gamma_values), figsize=(18, 5))

for ax, gamma in zip(axs, gamma_values):
    M = int(N / gamma)
    singular_values = []
    
    # Generate matrices and compute singular values
    for _ in range(num_Ts):
        A = generate_complex_matrix(N, M)
        svd_vals = np.linalg.svd(A, compute_uv=False)
        singular_values.extend(svd_vals)
    
    singular_values = np.array(singular_values)
    print(f'{gamma=}')
    print(f'min={singular_values.min():.2f}, max={singular_values.max():.2f}')
    
    # Plot histogram of singular values
    ax.hist(singular_values, bins=50, density=True, alpha=0.7, label="Empirical")
    
    # Plot theoretical Marchenko-Pastur distribution for singular values
    x_vals = np.linspace(0, max(2, (1 + np.sqrt(gamma)) + 0.5), 1000)
    mp_pdf = marchenko_pastur_svd_pdf(x_vals, gamma)
    ax.plot(x_vals, mp_pdf, 'r-', label="Theoretical")
    
    min_sv = np.abs(1 - np.sqrt(gamma))
    max_sv = (1 + np.sqrt(gamma))
    
    ax.axvline(min_sv, color='g', linestyle='--', label=f'Min SV: {min_sv:.2f}')
    ax.axvline(max_sv, color='b', linestyle='--', label=f'Max SV: {max_sv:.2f}')
    
    if gamma > 1:
        ax.plot([0], [(gamma - 1) / gamma], 'ro', markersize=10, label="Point mass at 0")
    
    ax.set_title(f"γ = {gamma}")
    ax.set_xlabel("Singular Value")
    ax.set_ylabel("Density")
    ax.legend()
    fig.show()

plt.tight_layout()

gamma=128
min=9.50, max=13.30
gamma=256
min=13.81, max=18.66


In [108]:
N = 256
M = 512
A = generate_complex_matrix(N, M)
v_in = 1/np.sqrt(M)*np.ones(M)
(np.abs(A@v_in)**2).sum()

0.45002438108227266